In [1]:
# !pip install -q "arize-phoenix" "arize[AutoEmbeddings, LLM_Evaluation]"

In [2]:
from arize.pandas.embeddings import EmbeddingGenerator, UseCases
from arize.pandas.generative.llm_evaluation import sacre_bleu, rouge
import pandas as pd
import phoenix as px
import numpy as np

/home/vishesh/anaconda3/envs/alibidetect/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-15 17:23:19.562780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 17:23:19.650884: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-15 17:23:20.031613: W tensorflow/compiler/xla/stream_executor/

In [12]:
perturbation = "pitchshift"
strength = 10
ref_path = "/home/vishesh/Desktop/datasets/fleurs/embeddings/train_decoder_embeddings.csv"
# cur_path = f"/home/vishesh/Desktop/datasets/fleurs/embeddings/perturbations/{perturbation}/test_results_{perturbation}_{strength}.csv"
cur_path = f"/home/vishesh/Desktop/datasets/fleurs/embeddings/test_decoder_embeddings.csv"

In [13]:
baseline_df = pd.read_csv(ref_path)
recent_df = pd.read_csv(cur_path)

baseline_df.drop_duplicates(subset=['Labels'], inplace=True)
recent_df.drop_duplicates(subset=['Labels'], inplace=True)

baseline_df = baseline_df.reset_index(drop=True)  # recommended when using EmbeddingGenerator.generate_embeddings
recent_df = recent_df.reset_index(drop=True)

In [14]:
def compute_rougeL_scores(df: pd.DataFrame) -> pd.Series:
    return rouge(
        response_col=df["Predictions"],
        references_col=df["Labels"],
        rouge_types=["rougeL"],
    )["rougeL"]


baseline_df["rougeL_score"] = compute_rougeL_scores(baseline_df)
recent_df["rougeL_score"] = compute_rougeL_scores(recent_df)

  arize.utils.logging | INFO | Loading metric: rouge
  arize.utils.logging | INFO | Computing rouge scores


100%|██████████████████████████████████████| 1461/1461 [00:01<00:00, 741.47it/s]

  arize.utils.logging | INFO | Loading metric: rouge


  arize.utils.logging | INFO | Computing rouge scores


100%|████████████████████████████████████████| 350/350 [00:00<00:00, 730.85it/s]


In [15]:
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.NLP.SUMMARIZATION,
    model_name="distilbert-base-uncased",
)
baseline_df["embeddings"] = generator.generate_embeddings(text_col=baseline_df["Predictions"])
recent_df["embeddings"] = generator.generate_embeddings(text_col=recent_df["Predictions"])

  arize.utils.logging | INFO | Downloading pre-trained model 'distilbert-base-uncased'
  arize.utils.logging | INFO | Downloading tokenizer for 'distilbert-base-uncased'
  arize.utils.logging | INFO | Generating embedding vectors


  arize.utils.logging | INFO | Generating embedding vectors


In [16]:
embedding_columns = [f"Embedding_{i}" for i in range (1,769)]
baseline_df["embeddings"] = baseline_df[embedding_columns].values.tolist()
recent_df["embeddings"] = recent_df[embedding_columns].values.tolist()

def convert_to_list(lst):
    return np.array(lst)

baseline_df['embeddings'] = baseline_df['embeddings'].apply(convert_to_list)
recent_df['embeddings'] = recent_df['embeddings'].apply(convert_to_list)

In [17]:
columns_to_del = [f"Embedding_{i}" for i in range (1,769)]
baseline_df.drop(columns=columns_to_del, inplace=True)
recent_df.drop(columns=columns_to_del, inplace=True)

In [18]:
schema = px.Schema(
    tag_column_names=[
        "rougeL_score",
        "Labels",
    ],
    response_column_names=px.EmbeddingColumnNames(
        vector_column_name="embeddings", raw_data_column_name="Predictions"
    ),
)

In [19]:
baseline_ds = px.Dataset(dataframe=baseline_df, schema=schema, name="baseline")
recent_ds = px.Dataset(dataframe=recent_df, schema=schema, name="recent")

In [20]:
session = px.launch_app(primary=recent_ds, reference=baseline_ds)

🌍 To view the Phoenix app in your browser, visit http://localhost:35059/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
